In [25]:
import numpy as np
import os
import geopandas as gpd
import pandas as pd
from scipy import stats as scipy_stats
import scipy.signal
import rasterio
import rasterio.features
from rasterio import MemoryFile
from affine import Affine
from rasterio.plot import show
from rasterio.plot import show_hist
import matplotlib.pyplot as plt
import rasterstats
from scipy.ndimage import gaussian_filter
from shapely.geometry import Polygon, shape, MultiPolygon, Point, MultiPoint
import shapely
import subprocess as sp

# Import libs
import sys
module_path = os.path.abspath(os.path.join('..', 'libs'))
if module_path not in sys.path:
    sys.path.append(module_path)
from s3utils import *

In [7]:
S3R = S3Helper('roof-index')

# Read necessary_data
flat = S3R.read_shp_from_s3_as_gpd("missoula/geospatial/contiguous_flat.zip").to_crs(6514)

In [29]:
min_x, min_y, max_x, max_y = flat.total_bounds

def make_random_pts(min_x, min_y, max_x, max_y, n_samples=10):
    
    out = []
    for n in range(n_samples):
        x = np.random.uniform(min_x, max_x)
        y = np.random.uniform(min_y, max_y)
        out.append(Point(x, y))
    
    return MultiPoint(out)

In [54]:
num_pts=10

pts = make_random_pts(min_x, min_y, max_x, max_y, num_pts)

def calc_distances(shp, pts):
    flat['centers'] = flat.centroid
    flat['distances'] = flat['centers'].apply(lambda x: {i: x.distance(y) for i, y in enumerate(pts)})
    
    return flat

calc_distances(flat, pts)

,fid,AVG_HEIGHT,MIN_LAG,area_sq_ft,fa_sq_ft,fa_perc,flat,area,geometry,flat_id,centers,distances
354,6.0,16.3807,3207.39,30646.00,5500.116176,17.947256,1,3612.995677,"POLYGON ((257446.577 300263.357, 257446.577 30...",0,POINT (257438.895 300250.356),"{0: 1327.719495677113, 1: 359.982971235251, 2:..."
351,6.0,16.3807,3207.39,30646.00,5500.116176,17.947256,1,1555.121217,"POLYGON ((257440.176 300299.933, 257440.176 30...",1,POINT (257441.287 300289.163),"{0: 1292.2519110447204, 1: 321.4444940657115, ..."
352,6.0,16.3807,3207.39,30646.00,5500.116176,17.947256,1,1520.998528,"POLYGON ((257426.460 300265.186, 257426.460 30...",2,POINT (257433.204 300259.707),"{0: 1322.063911848602, 1: 350.32485612970174, ..."
353,6.0,16.3807,3207.39,30646.00,5500.116176,17.947256,1,5030.995131,"POLYGON ((257423.717 300254.213, 257423.717 30...",3,POINT (257436.261 300245.418),"{0: 1333.3160953191443, 1: 364.7549606299562, ..."
355,6.0,16.3807,3207.39,30646.00,5500.116176,17.947256,1,1338.972555,"POLYGON ((257452.978 300235.010, 257452.978 30...",4,POINT (257450.860 300220.486),"{0: 1348.966833431485, 1: 390.6838601236398, 2..."
...,...,...,...,...,...,...,...,...,...,...,...,...
103,888.0,22.1743,3205.78,10648.10,1558.137650,14.633011,1,2015.998049,"POLYGON ((257967.785 301258.224, 257967.785 30...",389,POINT (257976.508 301250.766),"{0: 192.9452006631693, 1: 851.6592708940535, 2..."
104,888.0,22.1743,3205.78,10648.10,1558.137650,14.633011,1,1438.772499,"POLYGON ((257995.217 301250.909, 257995.217 30...",390,POINT (257994.099 301243.547),"{0: 193.93906191730971, 1: 857.9914321731763, ..."
122,908.0,16.6370,3218.74,16516.10,3283.610505,19.881270,1,4859.222280,"POLYGON ((258373.778 301198.788, 258373.778 30...",391,POINT (258377.724 301182.377),"{0: 411.0856426940942, 1: 1119.417905720349, 2..."
112,917.0,24.0490,3209.86,4434.73,894.283229,20.165449,1,1268.702875,"POLYGON ((258115.918 301266.454, 258115.918 30...",392,POINT (258113.067 301257.081),"{0: 183.85912361827505, 1: 951.4258034461348, ..."


In [47]:
min([x.distance(y) for x in pts for y in pts if x.distance(y) != 0])

66.44066821397683